In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
from array import array
from geojson import Feature, FeatureCollection, Point
import geojson
import json
%matplotlib inline

In [3]:
drive_node_gdf = gpd.read_file("C:/Users/wangs1/Documents/Met_Council/gtfs_transit_network_builder/data/output/drive_node_with_rail.shp")

In [5]:
walk_node_gdf = gpd.read_file("C:/Users/wangs1/Documents/Met_Council/centroid_connector_builder/walk_node.shp")
bike_node_gdf = gpd.read_file("C:/Users/wangs1/Documents/Met_Council/centroid_connector_builder/bike_node.shp")

In [6]:
drive_link_gdf = gpd.read_file("C:/Users/wangs1/Documents/Met_Council/gtfs_transit_network_builder/data/output/drive_and_rail_link.shp")

In [7]:
walk_link_gdf = gpd.read_file("C:/Users/wangs1/Documents/Met_Council/osm_roadway_network_builder/walk_link.shp")
bike_link_gdf = gpd.read_file("C:/Users/wangs1/Documents/Met_Council/osm_roadway_network_builder/bike_link.shp")

In [8]:
drive_node_gdf['isDriveNode'] = np.where(drive_node_gdf.OSMID > 0, 1, 0)
drive_node_gdf['isDedicatedTransitNode'] = np.where(drive_node_gdf.OSMID == 0, 1, 0)
walk_node_gdf['isPedNode'] = np.int(1)
bike_node_gdf['isBikeNode'] = np.int(1)

drive_link_gdf['isDriveLink'] = np.where(drive_link_gdf.LANES > 0, 1, 0)
drive_link_gdf['isDedicatedTransitLink'] = np.where(drive_link_gdf.LANES.isnull(), 1, 0)
walk_link_gdf['isPedLink'] = np.int(1)
bike_link_gdf['isBikeLink'] = np.int(1)

# faking id...
drive_node_gdf['id'] = drive_node_gdf.apply(lambda x: 'driveNode' + str(x.N), axis = 1)
bike_node_gdf['id'] = bike_node_gdf.apply(lambda x: 'bikeNode' + str(x.N), axis = 1)
walk_node_gdf['id'] = walk_node_gdf.apply(lambda x: 'walkNode' + str(x.N), axis = 1)

drive_node_gdf['travelModelId'] = drive_node_gdf['N']
bike_node_gdf['travelModelId'] = bike_node_gdf['N']
walk_node_gdf['travelModelId'] = walk_node_gdf['N']

# faking id...
drive_link_gdf['id'] = drive_link_gdf.apply(lambda x: 'driveLink' + str(x.LINK_ID), axis = 1)
bike_link_gdf['id'] = bike_link_gdf.apply(lambda x: 'bikeLink' + str(x.LINK_ID), axis = 1)
walk_link_gdf['id'] = walk_link_gdf.apply(lambda x: 'walkLink' + str(x.LINK_ID), axis = 1)

def link_AB_node(node, link):
    node_dict = dict(zip(node.N, node.id))
    link['A_id'] = link['A'].map(node_dict)
    link['B_id'] = link['B'].map(node_dict)
    
    return link

drive_link_gdf = link_AB_node(drive_node_gdf, drive_link_gdf)
walk_link_gdf = link_AB_node(walk_node_gdf, walk_link_gdf)
bike_link_gdf = link_AB_node(bike_node_gdf, bike_link_gdf)

# faking id...
drive_link_gdf['geometryId'] = drive_link_gdf.apply(lambda x: 'driveGeom' + str(x.LINK_ID), axis = 1)
bike_link_gdf['geometryId'] = bike_link_gdf.apply(lambda x: 'bikeGeom' + str(x.LINK_ID), axis = 1)
walk_link_gdf['geometryId'] = walk_link_gdf.apply(lambda x: 'walkGeom' + str(x.LINK_ID), axis = 1)

In [9]:
all_link_gdf = pd.concat([drive_link_gdf, bike_link_gdf, walk_link_gdf], sort = False, ignore_index = True)
all_node_gdf = pd.concat([drive_node_gdf, bike_node_gdf, walk_node_gdf], sort = False, ignore_index = True)

In [10]:
all_node_gdf_latlon = all_node_gdf.copy()
all_node_gdf_latlon = all_node_gdf_latlon.to_crs({'init': 'epsg:4326'})

In [11]:
all_link_gdf.crs
all_link_gdf_latlon = all_link_gdf.copy()
all_link_gdf_latlon = all_link_gdf_latlon.to_crs({'init': 'epsg:4326'})

In [13]:
def node_geojson(node, link):
    
    node_gdf = node.copy()
    
    link_gdf = link.copy()
    
    ob_grouped_df = link_gdf.groupby(['A_id'])
    ob_ref_df = pd.DataFrame({'outboundReferenceId' : ob_grouped_df['id'].apply(list)}).reset_index()
    ob_ref_dict = dict(zip(ob_ref_df.A_id, ob_ref_df.outboundReferenceId))
    node_gdf['outboundReferenceId'] = node_gdf['id'].map(ob_ref_dict)
    
    ib_grouped_df = link_gdf.groupby(['B_id'])
    ib_ref_df = pd.DataFrame({'inboundReferenceId' : ib_grouped_df['id'].apply(list)}).reset_index()
    ib_ref_dict = dict(zip(ib_ref_df.B_id, ib_ref_df.inboundReferenceId))
    node_gdf['inboundReferenceId'] = node_gdf['id'].map(ib_ref_dict)
        
    node_gdf.rename(columns = {'OSMID' : 'osmNodeId'}, inplace = True)
    
    node_gdf = node_gdf[(node_gdf.outboundReferenceId.notnull()) | (node_gdf.inboundReferenceId.notnull())]
    
    return node_gdf

In [14]:
def link_json(node, link):
    
    link_gdf = link.copy()

    node_gdf = node.copy()
    node_gdf['X'] = node_gdf['geometry'].apply(lambda p: p.x)
    node_gdf['Y'] = node_gdf['geometry'].apply(lambda p: p.y)
    node_gdf['point'] = [list(xy) for xy in zip(node_gdf.X, node_gdf.Y)]
    node_dict = dict(zip(node_gdf.id, node_gdf.point))
    
    link_gdf['A_point'] = link_gdf['A_id'].map(node_dict)
    link_gdf['B_point'] = link_gdf['B_id'].map(node_dict)
    link_gdf['locationReferences'] = link_gdf.apply(lambda x: [{'sequence':1, 
                                                             'point': x['A_point'],
                                                             'distanceToNextRef':x['DISTANCE']*1609.344,
                                                            'bearing' : 0,
                                                             'intersectionId':x['A_id']},
                                                                         {'sequence':2, 
                                                             'point': x['B_point'],
                                                             'intersectionId':x['B_id']}],
                                                   axis = 1)
     
    return link_gdf

In [15]:
def link_geojson(node, link):
    
    link_gdf = link.copy()
    
    link_gdf.rename(columns = {'A_id': 'fromIntersectionId', 
                               'B_id': 'toIntersectionId', 
                               'id' : 'forwardReferenceId'},
                   inplace = True)
    
    return link_gdf

In [16]:
# node geojson
all_node_geojson = node_geojson(all_node_gdf_latlon, all_link_gdf_latlon)

all_node_geojson.fillna({'isDriveNode' : 0,
                        'isDedicatedTransitNode' : 0,
                        'isBikeNode' : 0,
                        'isPedNode' : 0}, inplace = True)
all_node_geojson['isDriveNode'] = all_node_geojson['isDriveNode'].astype(int)
all_node_geojson['isDedicatedTransitNode'] = all_node_geojson['isDedicatedTransitNode'].astype(int)
all_node_geojson['isBikeNode'] = all_node_geojson['isBikeNode'].astype(int)
all_node_geojson['isPedNode'] = all_node_geojson['isPedNode'].astype(int)

all_node_geojson_copy = all_node_geojson.copy()
all_node_geojson_copy.inboundReferenceId.loc[all_node_geojson_copy.inboundReferenceId.isnull()] = all_node_geojson_copy.inboundReferenceId.loc[all_node_geojson_copy.inboundReferenceId.isnull()].apply(lambda x: [])
all_node_geojson_copy.outboundReferenceId.loc[all_node_geojson_copy.outboundReferenceId.isnull()] = all_node_geojson_copy.outboundReferenceId.loc[all_node_geojson_copy.outboundReferenceId.isnull()].apply(lambda x: [])

features = []
insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["X"],
                                                    X["Y"])),
                            properties= X.drop(['N', 'X', 'Y', 'geometry']).to_dict()))

all_node_geojson_copy.apply(insert_features, axis=1)

#with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/all_node.geojson', 'w') as f:
#    f.write("\n".join(map(str, features)))

#with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/all_node.geojson', 'w', encoding='utf8') as fp:
#    geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

#out = geojson.dumps(features, indent = 4)
#with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/node.geojson', 'w') as f:
#    f.write(out[1:-1])
    
    
out = geojson.dumps(geojson.FeatureCollection(features), indent = 4)
with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/node.geojson', 'w') as f:
    f.write(out)

C:\ProgramData\Anaconda3\envs\metC\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [17]:
# link json

all_link_json = link_json(all_node_gdf_latlon, all_link_gdf_latlon)

all_link_json.fillna(0, inplace = True)
all_link_json['isDriveLink'] = all_link_json['isDriveLink'].astype(int)
all_link_json['isDedicatedTransitLink'] = all_link_json['isDedicatedTransitLink'].astype(int)
all_link_json['isBikeLink'] = all_link_json['isBikeLink'].astype(int)
all_link_json['isPedLink'] = all_link_json['isPedLink'].astype(int)

df = pd.DataFrame(all_link_json.drop(['LINK_ID', 'A', 'B', 'A_id', 'B_id','geometry', 'A_point', 'B_point'], 
                                  axis = 1))

#with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/all_link.json', 'w') as f:
#    f.write(df.to_json(orient='records', lines=True))

out = json.dumps(json.loads(df.to_json(orient = 'records')), indent = 4)

with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/link.json', 'w') as f:
    f.write(out)

In [18]:
# link geojson

all_link_geojson = link_geojson(all_node_gdf_latlon, all_link_gdf_latlon)
all_link_geojson.rename(columns = {'geometryId' : 'id'}, inplace = True)
#all_link_geojson[['id' ,'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'geometry']].to_file('C:/Users/wangs1/Documents/Met_Council/Network Standard/all_link.geojson', 
#                                                                                                       driver = 'GeoJSON')

features = []
insert_features = lambda X: features.append(
            geojson.Feature(geometry=X.geometry,
                            properties= X[['id' ,'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId']].to_dict()))

all_link_geojson.apply(insert_features, axis=1)

#with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/all_link.geojson', 'w') as f:
#    f.write("\n".join(map(str, features)))

#out = geojson.dumps(features, indent = 4)
#with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/shape.geojson', 'w') as f:
#    f.write(out[1:-1])
    
out = geojson.dumps(geojson.FeatureCollection(features), indent = 4)
with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/shape.geojson', 'w') as f:
    f.write(out)

In [ ]:
#
#
# creating subset for downtown st pauls
poly_gdf = gpd.read_file("C:/Users/wangs1/Documents/Met_Council/Network Standard/poly2.shp")
poly_gdf = poly_gdf.to_crs(all_link_gdf.crs)
print(poly_gdf)

stpaul_link_gdf = all_link_gdf_latlon[all_link_gdf_latlon.within(poly_gdf.geometry[0])]
stpaul_node_gdf = all_node_gdf_latlon[all_node_gdf_latlon.within(poly_gdf.geometry[0])]

# node geojson
stpaul_node_geojson = node_geojson(stpaul_node_gdf, stpaul_link_gdf)

stpaul_node_geojson.fillna({'isDriveNode' : 0,
                        'isDedicatedTransitNode' : 0,
                        'isBikeNode' : 0,
                        'isPedNode' : 0}, inplace = True)
stpaul_node_geojson['isDriveNode'] = stpaul_node_geojson['isDriveNode'].astype(int)
stpaul_node_geojson['isDedicatedTransitNode'] = stpaul_node_geojson['isDedicatedTransitNode'].astype(int)
stpaul_node_geojson['isBikeNode'] = stpaul_node_geojson['isBikeNode'].astype(int)
stpaul_node_geojson['isPedNode'] = stpaul_node_geojson['isPedNode'].astype(int)

stpaul_node_geojson_copy = stpaul_node_geojson.copy()
stpaul_node_geojson_copy.inboundReferenceId.loc[stpaul_node_geojson_copy.inboundReferenceId.isnull()] = stpaul_node_geojson_copy.inboundReferenceId.loc[stpaul_node_geojson_copy.inboundReferenceId.isnull()].apply(lambda x: [])
stpaul_node_geojson_copy.outboundReferenceId.loc[stpaul_node_geojson_copy.outboundReferenceId.isnull()] = stpaul_node_geojson_copy.outboundReferenceId.loc[stpaul_node_geojson_copy.outboundReferenceId.isnull()].apply(lambda x: [])

features = []
insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["X"],
                                                    X["Y"])),
                            properties= X.drop(['N', 'X', 'Y', 'geometry']).to_dict()))

stpaul_node_geojson_copy.apply(insert_features, axis=1)

#with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/stpaul_node.geojson', 'w') as f:
#    f.write("\n".join(map(str, features)))

#with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/stpaul_node.geojson', 'w', encoding='utf8') as fp:
#    geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

#out = geojson.dumps(features, indent = 4)
#with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/stpaul_node.geojson', 'w') as f:
#    f.write(out[1:-1])

out = geojson.dumps(geojson.FeatureCollection(features), indent = 4)
with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/stpaul_node.geojson', 'w') as f:
    f.write(out)

    
# link json

stpaul_link_json = link_json(stpaul_node_gdf, stpaul_link_gdf)

stpaul_link_json.fillna(0, inplace = True)
stpaul_link_json['isDriveLink'] = stpaul_link_json['isDriveLink'].astype(int)
stpaul_link_json['isDedicatedTransitLink'] = stpaul_link_json['isDedicatedTransitLink'].astype(int)
stpaul_link_json['isBikeLink'] = stpaul_link_json['isBikeLink'].astype(int)
stpaul_link_json['isPedLink'] = stpaul_link_json['isPedLink'].astype(int)

df = pd.DataFrame(stpaul_link_json.drop(['LINK_ID', 'A', 'B', 'A_id', 'B_id','geometry', 'A_point', 'B_point'], 
                                  axis = 1))

#with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/stpaul_link.json', 'w') as f:
#    f.write(df.to_json(orient='records', lines=True))

out = json.dumps(json.loads(df.to_json(orient = 'records')), indent = 4)

with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/stpaul_link.json', 'w') as f:
    f.write(out)
    

# link geojson

stpaul_link_geojson = link_geojson(stpaul_node_gdf, stpaul_link_gdf)
stpaul_link_geojson.rename(columns = {'geometryId' : 'id'}, inplace = True)
#stpaul_link_geojson[['id' ,'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'geometry']].to_file('C:/Users/wangs1/Documents/Met_Council/Network Standard/stpaul_link.geojson', 
#                                                                                                       driver = 'GeoJSON')

features = []
insert_features = lambda X: features.append(
            geojson.Feature(geometry=X.geometry,
                            properties= X[['id' ,'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId']].to_dict()))

stpaul_link_geojson.apply(insert_features, axis=1)

#with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/stpaul_link.geojson', 'w') as f:
#    f.write("\n".join(map(str, features)))

#out = geojson.dumps(features, indent = 4)
#with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/stpaul_shape.geojson', 'w') as f:
#    f.write(out[1:-1])

out = geojson.dumps(geojson.FeatureCollection(features), indent = 4)
with open('C:/Users/wangs1/Documents/Met_Council/Network Standard/stpaul_shape.geojson', 'w') as f:
    f.write(out)